### Pre-installs

In [1]:
# %pip install pandas
# %pip install numpy
# %pip install matplotlib
# %pip install seaborn
# %pip install sklearn
# %pip install -U scikit-learn
# %pip install spotipy

### Inputs

In [2]:
input_song = input("Enter your desired song name here!")
input_playlist = input("Enter your desired playlist name here!s")
recc_num = int(input("Enter how many songs you want to be recommended!"))
client_id = input("Enter your Spotify API Client Key here!")
client_secret = input("Enter your Spotify API Secret Key here!")
sp_dc = input("Enter your Spotify sp_dc here!")
sp_key = input("Enter your Spotify sp_key here!")

### CSV & Spotify Information

In [3]:
import pandas as pd
import ast

# Importing the datasets
dataset = pd.read_csv("data/tracks_with_genres_&_language.csv")
df = pd.read_csv("data/tracks.csv")

# Creating the dataframe
df_generated = pd.DataFrame(dataset)
df_generated['genres'] = df_generated['genres'].apply(ast.literal_eval)

In [4]:
df_generated.iloc[:,3:19].head()

,duration_ms,explicit,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,160187,0,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.484,0.265,0,-11.101,1,0.0322,0.394,0.000000,0.149,0.285,113.564,3
1,159560,0,['6lH5PpuiMa5SpfjoIOlwCS'],2020-03-13,0.671,0.867,2,-2.706,1,0.0571,0.436,0.000000,0.139,0.839,120.689,4
2,141987,0,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.405,0.365,6,-10.226,0,0.0289,0.255,0.000005,0.163,0.588,104.536,4
3,157093,0,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.477,0.352,1,-14.165,1,0.0300,0.406,0.000000,0.122,0.478,106.773,4
4,187333,0,['1Mxqyy3pSjf8kZZL4QVxS0'],2018-05-04,0.319,0.201,7,-17.796,1,0.0623,0.887,0.000000,0.904,0.239,117.153,3


In [5]:
# Spotify API Authentication Information
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
# api key
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id,
                                               client_secret=client_secret,
                                               redirect_uri="http://localhost/",
                                               scope="playlist-modify-private",
                                               show_dialog=True,
                                               cache_path="token.txt"
                                               ))

In [6]:
import seaborn as sns
import matplotlib.pyplot as plt

# # Assuming df_generated is your DataFrame
# # Adjust this part based on your actual DataFrame structure
# # For example, you might need to select specific columns or rows
# # or convert categorical variables to numerical values for visualization

# df_generated.columns
# df_graph = df_generated[['popularity', 'duration_ms', 'explicit',
#        'danceability', 'energy', 'key', 'loudness',
#        'speechiness', 'acousticness', 'instrumentalness', 'liveness',
#        'valence', 'tempo', 'time_signature']]

# sns.set(style="white")  # Set the style of the plot

# # Create a heatmap using seaborn with annotated values for the entire correlation matrix
# plt.figure(figsize=(12, 10))  # Adjust the figure size as needed
# heatmap = sns.heatmap(df_graph.corr(numeric_only=True), annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)

# # Customize the appearance of the heatmap
# heatmap.set_title('Heatmap of df_generated', fontsize=16)

# plt.show()

### Processing the Input Song

In [7]:
# Using Spotify API to search for a song's information based on input and adding the necessary information in a DataFrame.
def search_track(track_name):
    # Search for the track
    results = sp.search(q=track_name, type='track')

    # Check if the track exists
    if results['tracks']['total'] > 0:
        # Get the first track from the results
        track = results['tracks']['items'][0]
        explicit = int(track["explicit"] == True)
        artists = []
        id_artists = []
        for i in range (len(track["artists"])):
            artists.append(track["artists"][i]["name"])
            id_artists.append(track["artists"][i]["id"])

        print(f'Found track: {track["name"]} by {track["artists"][0]["name"]} from the album {track["album"]["name"]}.')
        track_dict = {"id": track["id"], "name": track["name"], "popularity": track["popularity"], 
                      "duration_ms": track["duration_ms"], "explicit": explicit, "artists": str(artists),
                      "id_artists": str(id_artists), "release_date": track["album"]["release_date"]}
        return track_dict
    else:
        print('Track not found')
        return None

# Creating a Single Row DataFrame for the input song.
track_result = search_track(input_song)
td = pd.DataFrame(track_result, index=[0])
td

Found track: Perfect by Ed Sheeran from the album ÷ (Deluxe).


,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date
0,0tgVpDi06FyKpA1z0VMD4v,Perfect,90,263400,0,['Ed Sheeran'],['6eUKZXaKkcviH0Ku9w2n3V'],2017-03-03


In [8]:
# Obtaining Feature Data from song based on its song_id from previous function and adding them to a DataFrame.
def get_audio_features(track_result):
    song_id = track_result["id"]
    results = sp.audio_features(song_id)

    if results:
        return results[0]
    else:
        print(f'No audio features found for song ID: {song_id}')
        return None

audio_features = get_audio_features(track_result)
af = pd.DataFrame(audio_features, index=[0])
# Taking out the irrevelant features.
af_formatted = af.drop(["type", "id", "uri", "track_href", "analysis_url", "duration_ms"], axis=1)
# Merging both DataFrames to sync up with the dataset's layout.
td = pd.concat([td, af_formatted], axis=1)
td['year'] = td['release_date'].str.extract(r'(\d{4})').astype(int)

In [9]:
year_priority = td['year'][0]
year_priority

2017

In [10]:
import ast

# Converting string array into regular array.
td['artists'] = td['artists'].apply(ast.literal_eval)

td

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,year
0,0tgVpDi06FyKpA1z0VMD4v,Perfect,90,263400,0,[Ed Sheeran],['6eUKZXaKkcviH0Ku9w2n3V'],2017-03-03,0.599,0.448,...,-6.312,1,0.0232,0.163,0,0.106,0.168,95.05,3,2017


In [11]:
# Viewing the input song features.
td['artists_count'] = len(td['artists'][0])
td

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,year,artists_count
0,0tgVpDi06FyKpA1z0VMD4v,Perfect,90,263400,0,[Ed Sheeran],['6eUKZXaKkcviH0Ku9w2n3V'],2017-03-03,0.599,0.448,...,1,0.0232,0.163,0,0.106,0.168,95.05,3,2017,1


In [12]:
# Extracting the artists from the input artists array.
expanded_artists = td['artists'].apply(lambda x: pd.Series(x))

# Rename the columns
expanded_artists.columns = [f"artist_{i+1}" for i in range(len(expanded_artists.columns))]

# Concatenate the expanded columns with the original DataFrame
td = pd.concat([td, expanded_artists], axis=1)
td.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,year,artists_count,artist_1
0,0tgVpDi06FyKpA1z0VMD4v,Perfect,90,263400,0,[Ed Sheeran],['6eUKZXaKkcviH0Ku9w2n3V'],2017-03-03,0.599,0.448,...,0.0232,0.163,0,0.106,0.168,95.05,3,2017,1,Ed Sheeran


In [13]:
# Obtaining input artist(s) genre(s) through the Spotify API.
def get_artist_genres(artist_name):
    try:
        results = sp.search(q='artist:' + artist_name, type='artist')
        genres = results['artists']['items'][0]['genres']
        return genres if genres else []
    except IndexError:
        return []
    
for i in range(td['artists_count'][0]):
    td['genres' + str(i+1)] = td['artist_' + str(i+1)].apply(get_artist_genres)
    td['genres' + str(i+1)] = td['genres' + str(i+1)].fillna('[]')
td

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,acousticness,instrumentalness,liveness,valence,tempo,time_signature,year,artists_count,artist_1,genres1
0,0tgVpDi06FyKpA1z0VMD4v,Perfect,90,263400,0,[Ed Sheeran],['6eUKZXaKkcviH0Ku9w2n3V'],2017-03-03,0.599,0.448,...,0.163,0,0.106,0.168,95.05,3,2017,1,Ed Sheeran,"[pop, singer-songwriter pop, uk pop]"


In [14]:
# Combining all genres into one list.
for i in range(td['artists_count'][0]):
    td['genres'] = td['genres' + str(i+1)] + td['genres' + str(i+1)]
    td.drop(['genres' + str(i+1)], axis=1, inplace=True)
td

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,acousticness,instrumentalness,liveness,valence,tempo,time_signature,year,artists_count,artist_1,genres
0,0tgVpDi06FyKpA1z0VMD4v,Perfect,90,263400,0,[Ed Sheeran],['6eUKZXaKkcviH0Ku9w2n3V'],2017-03-03,0.599,0.448,...,0.163,0,0.106,0.168,95.05,3,2017,1,Ed Sheeran,"[pop, singer-songwriter pop, uk pop, pop, sing..."


In [15]:
# Returning only the unique genres for the input.
def get_unique(row):
    return list(set(row))

td['genres'] = td['genres'].apply(get_unique)

In [16]:
# Determining the language from a Lyrics Scraping Library.
import spotify_lyrics_scraper as spotify

token = spotify.getToken(sp_dc, sp_key)
lyrics_data = spotify.getLyrics(token, songName=td['name'][0])
if lyrics_data['status'] == False:
        language = "null"
else:
    # Extract the language from the lyrics data
    language = lyrics_data['message']['lyrics']['language']

td['language'] = language
td

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,instrumentalness,liveness,valence,tempo,time_signature,year,artists_count,artist_1,genres,language
0,0tgVpDi06FyKpA1z0VMD4v,Perfect,90,263400,0,[Ed Sheeran],['6eUKZXaKkcviH0Ku9w2n3V'],2017-03-03,0.599,0.448,...,0,0.106,0.168,95.05,3,2017,1,Ed Sheeran,"[singer-songwriter pop, uk pop, pop]",en


In [17]:
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException

# Many songs do not have lyrics enabled by spotify. So the alternative to that is to use their name as a benchmark for the language.
# Uses language detect library to identify the song name's expected language. Not exactly accurate but it is an acceptable alternative for now until NLP.
def replace_nan_with_language(row):
    if pd.isna(row['language']):  # Check if the value is NaN
        name = row['name']
        if isinstance(name, str) and len(name) > 0:  # Check if name is a non-empty string
            try:
                return detect(name)  # Use detect() output as the value if it's NaN
            except LangDetectException:
                pass  # Handle LangDetectException, e.g., return 'unknown' or None
        # Return None or any other default value if language detection fails or input text is empty
        return None  
    else:
        return row['language']  # Otherwise, keep the original value

td['language'] = td.apply(replace_nan_with_language, axis=1)

td.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,instrumentalness,liveness,valence,tempo,time_signature,year,artists_count,artist_1,genres,language
0,0tgVpDi06FyKpA1z0VMD4v,Perfect,90,263400,0,[Ed Sheeran],['6eUKZXaKkcviH0Ku9w2n3V'],2017-03-03,0.599,0.448,...,0,0.106,0.168,95.05,3,2017,1,Ed Sheeran,"[singer-songwriter pop, uk pop, pop]",en


In [18]:
# Creating a unique identifier.
td['song_name_artist'] = td['name'] + str(td['artists'])

In [19]:
# Defining the values to give priority to later.
language_prority = td['language'][0]
key_priority = td['key'][0]
genre_priority = td['genres'][0]

### Adding Input to Dataset and further processing

In [20]:
# Adding the Input Song to the Dataset DataFrame. Added to the very front.
new_df = pd.concat([td, df_generated], ignore_index=True)
new_df.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,time_signature,year,artists_count,artist_1,genres,language,song_name_artist,artist_2,artist_3,artist_4
0,0tgVpDi06FyKpA1z0VMD4v,Perfect,90,263400,0,[Ed Sheeran],['6eUKZXaKkcviH0Ku9w2n3V'],2017-03-03,0.599,0.448,...,3,2017,1,Ed Sheeran,"[singer-songwriter pop, uk pop, pop]",en,"Perfect0 [Ed Sheeran]\nName: artists, dtype...",NaN,NaN,NaN
1,6catF1lDhNTjjGa2GxRQNN,You'll Never Walk Alone - Mono; 2002 Remaster,56,160187,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.484,0.265,...,3,2008,1,Gerry & The Pacemakers,"[adult standards, bubblegum pop, merseybeat, r...",en,You'll Never Walk Alone - Mono; 2002 Remaster[...,NaN,NaN,NaN
2,6Pkt6qVikqPBt9bEQy8iTz,A Lover's Concerto,41,159560,0,NaN,['6lH5PpuiMa5SpfjoIOlwCS'],2020-03-13,0.671,0.867,...,4,2020,1,The Toys,[],en,A Lover's Concerto['The Toys'],NaN,NaN,NaN
3,4aSw1QJIMwYSoDEgzgdCJL,Ferry Cross the Mersey - Mono; 2002 Remaster,40,141987,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.405,0.365,...,4,2008,1,Gerry & The Pacemakers,"[adult standards, bubblegum pop, merseybeat, r...",en,Ferry Cross the Mersey - Mono; 2002 Remaster['...,NaN,NaN,NaN
4,0ZMMtH875IR2TfkyC4PolD,Don't Let the Sun Catch You Crying (Main) - Mono,34,157093,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.477,0.352,...,4,2008,1,Gerry & The Pacemakers,"[adult standards, bubblegum pop, merseybeat, r...",en,Don't Let the Sun Catch You Crying (Main) - Mo...,NaN,NaN,NaN


In [21]:
new_df['langauge'] = new_df['language'].fillna("null")
new_df.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,year,artists_count,artist_1,genres,language,song_name_artist,artist_2,artist_3,artist_4,langauge
0,0tgVpDi06FyKpA1z0VMD4v,Perfect,90,263400,0,[Ed Sheeran],['6eUKZXaKkcviH0Ku9w2n3V'],2017-03-03,0.599,0.448,...,2017,1,Ed Sheeran,"[singer-songwriter pop, uk pop, pop]",en,"Perfect0 [Ed Sheeran]\nName: artists, dtype...",NaN,NaN,NaN,en
1,6catF1lDhNTjjGa2GxRQNN,You'll Never Walk Alone - Mono; 2002 Remaster,56,160187,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.484,0.265,...,2008,1,Gerry & The Pacemakers,"[adult standards, bubblegum pop, merseybeat, r...",en,You'll Never Walk Alone - Mono; 2002 Remaster[...,NaN,NaN,NaN,en
2,6Pkt6qVikqPBt9bEQy8iTz,A Lover's Concerto,41,159560,0,NaN,['6lH5PpuiMa5SpfjoIOlwCS'],2020-03-13,0.671,0.867,...,2020,1,The Toys,[],en,A Lover's Concerto['The Toys'],NaN,NaN,NaN,en
3,4aSw1QJIMwYSoDEgzgdCJL,Ferry Cross the Mersey - Mono; 2002 Remaster,40,141987,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.405,0.365,...,2008,1,Gerry & The Pacemakers,"[adult standards, bubblegum pop, merseybeat, r...",en,Ferry Cross the Mersey - Mono; 2002 Remaster['...,NaN,NaN,NaN,en
4,0ZMMtH875IR2TfkyC4PolD,Don't Let the Sun Catch You Crying (Main) - Mono,34,157093,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.477,0.352,...,2008,1,Gerry & The Pacemakers,"[adult standards, bubblegum pop, merseybeat, r...",en,Don't Let the Sun Catch You Crying (Main) - Mo...,NaN,NaN,NaN,en


In [22]:
new_df['song_id_name'] = new_df['id'] + new_df['name']
print(new_df.shape)
new_df.drop_duplicates(subset=["song_id_name"], keep='first', inplace=True)
print(new_df.shape)

(186012, 31)
(186011, 31)


In [23]:
# Counting the number of genres in for each song.
def count_items_in_list(lst):
    return len(lst)

# Apply the function to the DataFrame column
new_df['genres_count'] = new_df['genres'].apply(count_items_in_list)
new_df.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,artist_1,genres,language,song_name_artist,artist_2,artist_3,artist_4,langauge,song_id_name,genres_count
0,0tgVpDi06FyKpA1z0VMD4v,Perfect,90,263400,0,[Ed Sheeran],['6eUKZXaKkcviH0Ku9w2n3V'],2017-03-03,0.599,0.448,...,Ed Sheeran,"[singer-songwriter pop, uk pop, pop]",en,"Perfect0 [Ed Sheeran]\nName: artists, dtype...",NaN,NaN,NaN,en,0tgVpDi06FyKpA1z0VMD4vPerfect,3
1,6catF1lDhNTjjGa2GxRQNN,You'll Never Walk Alone - Mono; 2002 Remaster,56,160187,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.484,0.265,...,Gerry & The Pacemakers,"[adult standards, bubblegum pop, merseybeat, r...",en,You'll Never Walk Alone - Mono; 2002 Remaster[...,NaN,NaN,NaN,en,6catF1lDhNTjjGa2GxRQNNYou'll Never Walk Alone ...,9
2,6Pkt6qVikqPBt9bEQy8iTz,A Lover's Concerto,41,159560,0,NaN,['6lH5PpuiMa5SpfjoIOlwCS'],2020-03-13,0.671,0.867,...,The Toys,[],en,A Lover's Concerto['The Toys'],NaN,NaN,NaN,en,6Pkt6qVikqPBt9bEQy8iTzA Lover's Concerto,0
3,4aSw1QJIMwYSoDEgzgdCJL,Ferry Cross the Mersey - Mono; 2002 Remaster,40,141987,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.405,0.365,...,Gerry & The Pacemakers,"[adult standards, bubblegum pop, merseybeat, r...",en,Ferry Cross the Mersey - Mono; 2002 Remaster['...,NaN,NaN,NaN,en,4aSw1QJIMwYSoDEgzgdCJLFerry Cross the Mersey -...,9
4,0ZMMtH875IR2TfkyC4PolD,Don't Let the Sun Catch You Crying (Main) - Mono,34,157093,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.477,0.352,...,Gerry & The Pacemakers,"[adult standards, bubblegum pop, merseybeat, r...",en,Don't Let the Sun Catch You Crying (Main) - Mo...,NaN,NaN,NaN,en,0ZMMtH875IR2TfkyC4PolDDon't Let the Sun Catch ...,9


In [24]:
print(new_df.shape)
new_df = new_df[(new_df['genres_count'] != 0)]
print(new_df.shape)

(186011, 32)
(166251, 32)


### Supersetting genres

In [25]:
# Determining the frequency of the genre. Giving the input song's genre priority to go first.
genre_extract = new_df['genres'].reset_index()
genre_wo_index = genre_extract['genres']
tally = genre_wo_index.count()
genre_dict = {}

for i in range (tally):
    input = genre_wo_index.loc[i]
    if len(input) > 0:
        for n in input:
            if n in genre_dict.keys():
                genre_dict[n] += 1
            else:
                genre_dict[n] = 1

print(genre_dict)

{'singer-songwriter pop': 1, 'uk pop': 681, 'pop': 6880, 'adult standards': 1216, 'bubblegum pop': 416, 'merseybeat': 529, 'rock-and-roll': 650, 'british invasion': 495, 'classic uk pop': 704, 'brill building pop': 763, 'rockabilly': 550, 'folk rock': 933, 'easy listening': 126, 'lounge': 283, 'british blues': 265, 'canadian blues': 54, 'singer-songwriter': 312, 'lilith': 307, 'canadian singer-songwriter': 107, 'folk': 393, 'rock': 3193, 'mellow gold': 1141, 'jazz': 304, 'swing': 56, 'big band': 94, 'jazz piano': 73, 'stride': 75, 'vocal jazz': 355, 'cool jazz': 274, 'harlem renaissance': 23, 'jazz saxophone': 71, 'contemporary jazz': 73, 'jazz quartet': 20, 'jazz fusion': 297, 'swedish jazz': 134, 'free jazz': 58, 'avant-garde jazz': 49, 'contemporary post-bop': 57, 'italian jazz': 15, 'soundtrack': 456, 'italian soundtrack': 28, 'vintage italian soundtrack': 34, 'classic soundtrack': 89, 'jazz trumpet': 176, 'hard bop': 204, 'roots rock': 250, 'psychedelic rock': 592, 'classic rock':

In [26]:
genres_df = pd.Series(genre_dict)
genres_df.head()

singer-songwriter pop       1
uk pop                    681
pop                      6880
adult standards          1216
bubblegum pop             416
dtype: int64

In [27]:
# Summary Statistics of genres.
genres_df.T.describe()

count    4230.000000
mean      130.101655
std       393.268710
min         1.000000
25%         4.000000
50%        19.000000
75%        85.000000
max      6880.000000
dtype: float64

In [28]:
# genres_df.to_csv("data/genre_frequency.csv")

In [29]:
# Creating the genre supersets, with priority to the input song's genre.
# Since the input song is the first one to be run, its genre will be able to start the classification of other sub-genres under it.

# This can be further improved. Allowing for later genres to take over earlier genres as the superset if it more general.
# E.g.: Detroit Hip Hop is found before general hip hop. Currently they will be seperated into 2 seperate genres because the first word 'detroit' is
# not 'hip hop' and is thus overlooked.

def generate_superset_mapping(genre_dict):
    superset_mapping = {}

    for genre, frequency in genre_dict.items():
        subset_added = False
        for superset in list(superset_mapping.keys()):
            if genre in superset:
                superset_mapping[genre] = [genre]
                superset_mapping[genre].extend(superset_mapping[superset])
                del superset_mapping[superset]
                subset_added = True
                break
            elif superset in genre:
                if superset in superset_mapping:
                    superset_mapping[superset].append(genre)
                    subset_added = True
                    break
        if not subset_added:
            superset_mapping[genre] = [genre]

    return superset_mapping

# Generate superset mapping
superset_mapping = generate_superset_mapping(genre_dict)

# Output the superset mapping
for superset, subsets in superset_mapping.items():
    print(f"{superset}: {subsets}")


uk pop: ['uk pop', 'classic uk pop', 'uk pop punk']
pop: ['pop', 'singer-songwriter pop', 'bubblegum pop', 'brill building pop', 'sunshine pop', 'baroque pop', 'pop rock', 'dance pop', 'post-teen pop', 'pop rap', 'new wave pop', 'finnish dance pop', 'finnish pop', 'europop', 'canadian pop', 'barbadian pop', 'latin pop', 'acoustic pop', 'spanish pop rock', 'mexican pop', 'latin arena pop', 'spanish pop', 'bow pop', 'pop nacional', 'socal pop punk', 'britpop', 'sophisti-pop', 'hip pop', 'south african pop', 'south african pop dance', 'bahamian pop', 'puerto rican pop', 'power-pop punk', 'deep power-pop punk', 'canadian pop punk', 'antiviral pop', 'j-pop girl group', 'australian pop', 'art pop', 'indie pop', 'pop punk', 'synthpop', 'norwegian pop', 'candy pop', 'ambient pop', 'glitch pop', 'french pop', 'dream pop', 'italian adult pop', 'indonesian pop', 'electropop', 'german pop', 'german pop rock', 'tatar pop', 'collage pop', 'pop r&b', 'belarusian pop', 'pop rock brasileiro', 'c-pop', 

In [30]:
# Supersetting the weighted genres for later.

superset_genre_priority = []
if len(genre_priority) != 0:
    for i in genre_priority:
        for n in superset_mapping:
            if i in n:
                superset_genre_priority.append(n)
print(superset_genre_priority)

['uk pop', 'uk pop', 'pop']


In [31]:
# Now with the superset mapped out, apply it onto the all the songs to determine their supersetted genre(s).
# This can be improved. Instead of only returning the first superset genre, it should return multiple in an array.

import pandas as pd

new_df['supersetted_genres'] = new_df['genres'].apply(lambda genres: [m for n in genres for m in superset_mapping if n in m])

In [32]:
new_df['supersetted_genres'].value_counts().count()

6075

In [33]:
new_df.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,genres,language,song_name_artist,artist_2,artist_3,artist_4,langauge,song_id_name,genres_count,supersetted_genres
0,0tgVpDi06FyKpA1z0VMD4v,Perfect,90,263400,0,[Ed Sheeran],['6eUKZXaKkcviH0Ku9w2n3V'],2017-03-03,0.599,0.448,...,"[singer-songwriter pop, uk pop, pop]",en,"Perfect0 [Ed Sheeran]\nName: artists, dtype...",NaN,NaN,NaN,en,0tgVpDi06FyKpA1z0VMD4vPerfect,3,"[uk pop, uk pop, pop]"
1,6catF1lDhNTjjGa2GxRQNN,You'll Never Walk Alone - Mono; 2002 Remaster,56,160187,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.484,0.265,...,"[adult standards, bubblegum pop, merseybeat, r...",en,You'll Never Walk Alone - Mono; 2002 Remaster[...,NaN,NaN,NaN,en,6catF1lDhNTjjGa2GxRQNNYou'll Never Walk Alone ...,9,"[adult standards, merseybeat, british invasion..."
3,4aSw1QJIMwYSoDEgzgdCJL,Ferry Cross the Mersey - Mono; 2002 Remaster,40,141987,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.405,0.365,...,"[adult standards, bubblegum pop, merseybeat, r...",en,Ferry Cross the Mersey - Mono; 2002 Remaster['...,NaN,NaN,NaN,en,4aSw1QJIMwYSoDEgzgdCJLFerry Cross the Mersey -...,9,"[adult standards, merseybeat, british invasion..."
4,0ZMMtH875IR2TfkyC4PolD,Don't Let the Sun Catch You Crying (Main) - Mono,34,157093,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.477,0.352,...,"[adult standards, bubblegum pop, merseybeat, r...",en,Don't Let the Sun Catch You Crying (Main) - Mo...,NaN,NaN,NaN,en,0ZMMtH875IR2TfkyC4PolDDon't Let the Sun Catch ...,9,"[adult standards, merseybeat, british invasion..."
5,1hx7X9cMXHWJjknb9O6Ava,The September Of My Years - Live At The Sands ...,26,187333,0,NaN,['1Mxqyy3pSjf8kZZL4QVxS0'],2018-05-04,0.319,0.201,...,"[easy listening, adult standards, lounge]",en,The September Of My Years - Live At The Sands ...,NaN,NaN,NaN,en,1hx7X9cMXHWJjknb9O6AvaThe September Of My Year...,3,"[easy listening, adult standards, lounge]"


### Encoding

In [34]:
# # One-hot Encoding of all the superset genres of each song in the dataframe.

# genres_dummies = pd.get_dummies(new_df['supersetted_genres'].apply(pd.Series).stack(), prefix="genre").groupby(level=0).sum()

# # Concatenate the binary columns with the original DataFrame
# df_2 = pd.concat([new_df, genres_dummies], axis=1)

In [35]:
# df_2.iloc[:,0:20].describe()

In [36]:
for i in new_df.loc[0]['artists']:
    for index, row in new_df.iterrows():
        # Check if the value of x is in any of the artist columns
        if i in [row['artist_1'], row['artist_2'], row['artist_3'], row['artist_4']]:
            # If x is found in any of the artist columns, set the value of matching_artist column to 1
            new_df.at[index, 'matching_artist'] = 1
        else:
            new_df.at[index, 'matching_artist'] = 0
new_df['matching_artist'].head()

0    1.0
1    0.0
3    0.0
4    0.0
5    0.0
Name: matching_artist, dtype: float64

In [37]:
# Determining how many keys there are.
new_df.groupby('key').count()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,language,song_name_artist,artist_2,artist_3,artist_4,langauge,song_id_name,genres_count,supersetted_genres,matching_artist
key,,,,,,,,,,,,,,,,,,,,,
0,19586,19586,19586,19586,19586,0,19586,19586,19586,19586,...,19535,19586,4082,1013,194,19586,19586,19586,19586,19586
1,14502,14502,14502,14502,14502,0,14502,14502,14502,14502,...,14446,14502,3955,1026,220,14502,14502,14502,14502,14502
2,17337,17337,17337,17337,17337,0,17337,17337,17337,17337,...,17297,17337,3440,921,164,17337,17337,17337,17337,17337
3,5024,5024,5024,5024,5024,0,5024,5024,5024,5024,...,5004,5024,1090,271,37,5024,5024,5024,5024,5024
4,13626,13626,13626,13626,13626,0,13626,13626,13626,13626,...,13586,13626,2714,646,126,13626,13626,13626,13626,13626
5,13199,13199,13199,13199,13199,0,13199,13199,13199,13199,...,13165,13199,2882,709,162,13199,13199,13199,13199,13199
6,11252,11252,11252,11252,11252,0,11252,11252,11252,11252,...,11208,11252,2821,708,152,11252,11252,11252,11252,11252
7,19669,19669,19669,19669,19669,0,19669,19669,19669,19669,...,19618,19669,4162,1053,215,19669,19669,19669,19669,19669
8,10079,10079,10079,10079,10079,1,10079,10079,10079,10079,...,10042,10079,2474,646,140,10079,10079,10079,10079,10079


In [38]:
new_df['key'].max()

11

In [39]:
# One-hot encoding the keys.
key_encoded = pd.get_dummies(new_df['key'], columns=[i for i in range(12)], prefix='key').astype(int)

In [40]:
key_encoded.head()

,key_0,key_1,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11
0,0,0,0,0,0,0,0,0,1,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,1,0,0,0,0


In [41]:
# Adding keys encoding into the dataset.
df_2 = pd.concat([new_df, key_encoded], axis=1)

In [42]:
print(df_2.shape)
df_2.drop_duplicates(subset=["song_id_name"], keep='first', inplace=True)
print(df_2.shape)

(166251, 46)
(166251, 46)


In [43]:
# # Checking all the languages.
# df_2['langauge'].value_counts().keys()

In [44]:
# df_2['language'].head()

In [45]:
# # Encoding all the languages.
# df_2['language'] = df_2['language'].fillna('null')
# language_encoded = pd.get_dummies(df_2['language'], prefix='lang').astype(int)
# language_encoded.head()

In [46]:
# language_encoded = language_encoded.drop(['lang_null'], axis=1)
# language_encoded.head()

In [47]:
# # Adding language encoding into the dataset.
# df_2 = pd.concat([df_2, language_encoded], axis=1)

In [48]:
# print(df_2.shape)
# df_2.drop_duplicates(subset=["song_id_name"], keep='first', inplace=True)
# print(df_2.shape)

In [49]:
# Removing unnecessary features.
df_2 = df_2.drop(['id_artists', 'release_date', 'popularity', 'key', 'artists_count', 'song_name_artist', 'artist_1', 'artist_2', 'artist_3', 'artist_4', 'genres', 'explicit', 'time_signature'], axis=1)
df_2.head()

,id,name,duration_ms,artists,danceability,energy,loudness,mode,speechiness,acousticness,...,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11
0,0tgVpDi06FyKpA1z0VMD4v,Perfect,263400,[Ed Sheeran],0.599,0.448,-6.312,1,0.0232,0.163,...,0,0,0,0,0,0,1,0,0,0
1,6catF1lDhNTjjGa2GxRQNN,You'll Never Walk Alone - Mono; 2002 Remaster,160187,NaN,0.484,0.265,-11.101,1,0.0322,0.394,...,0,0,0,0,0,0,0,0,0,0
3,4aSw1QJIMwYSoDEgzgdCJL,Ferry Cross the Mersey - Mono; 2002 Remaster,141987,NaN,0.405,0.365,-10.226,0,0.0289,0.255,...,0,0,0,0,1,0,0,0,0,0
4,0ZMMtH875IR2TfkyC4PolD,Don't Let the Sun Catch You Crying (Main) - Mono,157093,NaN,0.477,0.352,-14.165,1,0.0300,0.406,...,0,0,0,0,0,0,0,0,0,0
5,1hx7X9cMXHWJjknb9O6Ava,The September Of My Years - Live At The Sands ...,187333,NaN,0.319,0.201,-17.796,1,0.0623,0.887,...,0,0,0,0,0,1,0,0,0,0


In [50]:
# Filling in any null values. (In case)

df_2 = df_2.fillna(0)
df_2.head()

,id,name,duration_ms,artists,danceability,energy,loudness,mode,speechiness,acousticness,...,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11
0,0tgVpDi06FyKpA1z0VMD4v,Perfect,263400,[Ed Sheeran],0.599,0.448,-6.312,1,0.0232,0.163,...,0,0,0,0,0,0,1,0,0,0
1,6catF1lDhNTjjGa2GxRQNN,You'll Never Walk Alone - Mono; 2002 Remaster,160187,0,0.484,0.265,-11.101,1,0.0322,0.394,...,0,0,0,0,0,0,0,0,0,0
3,4aSw1QJIMwYSoDEgzgdCJL,Ferry Cross the Mersey - Mono; 2002 Remaster,141987,0,0.405,0.365,-10.226,0,0.0289,0.255,...,0,0,0,0,1,0,0,0,0,0
4,0ZMMtH875IR2TfkyC4PolD,Don't Let the Sun Catch You Crying (Main) - Mono,157093,0,0.477,0.352,-14.165,1,0.0300,0.406,...,0,0,0,0,0,0,0,0,0,0
5,1hx7X9cMXHWJjknb9O6Ava,The September Of My Years - Live At The Sands ...,187333,0,0.319,0.201,-17.796,1,0.0623,0.887,...,0,0,0,0,0,1,0,0,0,0


In [51]:
df_2.shape

(166251, 33)

### Weighting

In [52]:
numericals = {'danceability': 0, 'energy': 0.15, 'loudness': 0.1, 'mode': 0.10, 'speechiness': 0.05, 'acousticness': 0.20, 
    'instrumentalness': 0.25, 'liveness': 0.05, 'valence': 0.15, 'tempo': 0.1}
categoricals = {'key_': 0.05}
numericals_key = numericals.keys()
categoricals_key = categoricals.keys()

In [53]:
from sklearn.preprocessing import MinMaxScaler
# Normalize numerical features
numerical_features = ['loudness', 'tempo']
scaler = MinMaxScaler()
df_2[numerical_features] = scaler.fit_transform(df_2[numerical_features])

# Standardize Year
df_2['standardized_year'] = scaler.fit_transform(df_2[['year']])


In [54]:
for i in numericals_key:
    df_2[i] = df_2[i] * numericals[i]

In [55]:
for n in categoricals_key:
    for z in df_2.columns:
        if n in z:
            df_2[z] = df_2[z] * categoricals[n]

In [56]:
# # Applying weight to input language.
# df_2['lang_' + language_prority] = df_2['lang_' + language_prority] * 10
# df_2[['name','lang_' + language_prority]].head()

In [57]:
# # Reducing weight from non-input language.
# for i in df_2.columns:
#     if 'lang_' in i:
#         if i != ('lang_' + language_prority):
#             df_2[i] = df_2[i] / 10
# df_2.head()

In [58]:
# # Applying weight to input key.
# df_2['key_' + str(key_priority)] = df_2['key_' + str(key_priority)] * 10
# df_2[['name','key_' + str(key_priority)]].head()

In [59]:
# # Reducing weight from non-input key.
# for i in df_2.columns:
#     if 'key_' in i:
#         if i != ('key_' + str(key_priority)):
#             df_2[i] = df_2[i] / 10
# df_2['key_7'].head()

In [60]:
# # Applying weight to input genre(s).
# superset_genre_priority = list(set(superset_genre_priority))

# if len(superset_genre_priority) != 0:
#     for i in superset_genre_priority:
#         df_2[i] = df_2[i] * 10
# df_2[superset_genre_priority].head()

In [61]:
# # Reducing weight from non-input genre(s).
# for n in superset_genre_priority:
#     for i in df_2.columns:
#         if i in superset_mapping.keys():
#             if i != (n):
#                 df_2[i] = df_2[i] / 10
# df_2['pop'].head()

### Clustering & K Nearest Neighbours

In [62]:
df_features = df_2[['energy', 'valence', 'acousticness']]

In [63]:
from sklearn.cluster import KMeans

# Kmeans model
kmeans = KMeans(n_clusters = 3, random_state = 300, n_init = 20, tol = 0.0001)
# Fit and predict on the data
y_kmeans = kmeans.fit_predict(df_features)
#Save centroids
centroids = kmeans.cluster_centers_
# Save the predictions as a column
df_2['y_kmeans'] = y_kmeans
# Check the distribution
df_2['y_kmeans'].value_counts()

y_kmeans
2    63525
0    55207
1    47519
Name: count, dtype: int64

In [64]:
from sklearn.neighbors import KNeighborsClassifier

neigh_X = df_2[['danceability', 'energy', 'loudness',
       'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo']]
neigh_y = df_2['y_kmeans']

neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(neigh_X, neigh_y)

cluster_values = td[['danceability', 'energy', 'loudness',
       'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo']]
print(neigh.predict(cluster_values))
print(neigh.predict_proba(cluster_values))

cluster_neigh = neigh.predict(cluster_values)[0]

[0]
[[0.66666667 0.33333333 0.        ]]


In [65]:
cluster_priority = df_2.loc[0]['y_kmeans']
cluster_priority

0

### Cosine Similarity & Output Extraction

In [66]:
import numpy as np
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

# Feature selection, removing non-useful rows.
def processing(df_2):
    # Create a feature matrix
    feature_matrix = df_2.drop(['id', 'name', 'artists', 'song_id_name', 'language', 'langauge', 'year', 'supersetted_genres', 'y_kmeans'], axis=1)

    # Apply normalization to the entire feature matrix
    feature_matrix = scaler.fit_transform(feature_matrix)

    # This is the input song. We are isolating it from the dataframe first.
    input_song = feature_matrix[0]
    
    arr = []

    # Returns comparison value of first song in dataset to all others in an array.
    # DataFrame can be customized before Feature Selection to change which song the subject of comparison should be.
    for i in range(1, len(feature_matrix)):
        # Take the i-th song from the feature matrix
        compare_song = feature_matrix[i]
        # Perform cosine similarity.
        cosine_sim = cosine_similarity([input_song], [compare_song])
        cs_value = cosine_sim[0][0]  # Take the single similarity value between the input song and the i-th song.
        arr.append(cs_value)

    return arr

arr = processing(df_2)
arr

[0.6045089059799409,
 0.4277584603703598,
 0.5997408129609468,
 0.5739296852257326,
 0.4105639990689198,
 0.5895864927508939,
 0.589735964206418,
 0.5821371374863046,
 0.4116646391082569,
 0.5828160980348995,
 0.586836090008554,
 0.5761802141132362,
 0.5884153603146731,
 0.5704130917893415,
 0.592846086063164,
 0.606548586065802,
 0.5919946929118995,
 0.608194154213981,
 0.4381202092683868,
 0.4271408620126166,
 0.4337462402881682,
 0.5904765786456792,
 0.5869358034803613,
 0.4310435507860084,
 0.5734768364397752,
 0.46058232618347816,
 0.5744508530459671,
 0.5036147016707724,
 0.37027172739775444,
 0.5135478825885629,
 0.39875414678355003,
 0.48793315629241035,
 0.5841631074403533,
 0.6024204239510005,
 0.5849150419230069,
 0.4528004168844747,
 0.6478348432836405,
 0.5837965570735897,
 0.5800899953306231,
 0.5899201053712942,
 0.42745109662589986,
 0.5885267446811406,
 0.648582105065745,
 0.4828905588786251,
 0.6456709048552536,
 0.43701293400029395,
 0.5816713454309117,
 0.4210281920

In [67]:
max(arr[1:])

0.987648961213725

In [68]:
def compile_suggestions(arr):
    suggestions = []
    
    # Compiling all song ids that are above the pre-determined threshold for 'like the first song'.
    for i in range(len(arr)):
        if arr[i] > 0.86:
            suggestions.append(i)
    
    print(suggestions)
    return suggestions
suggestions = compile_suggestions(arr)

[3127, 7547, 7624, 7959, 8198, 8212, 8622, 8879, 8894, 9307, 9681, 9686, 9758, 9912, 10046, 10058, 10208, 10406, 10414, 10575, 10845, 10944, 10972, 11037, 11099, 11101, 11170, 11197, 11221, 11231, 11264, 11272, 11285, 11306, 11329, 11347, 11365, 11449, 11479, 11552, 11792, 11799, 11808, 11901, 11952, 11987, 12038, 12071, 12137, 12160, 12187, 12196, 12381, 12395, 12539, 12598, 12643, 12741, 12934, 12990, 12996, 13004, 13069, 13073, 13101, 13107, 13144, 13250, 13280, 13308, 13371, 13495, 13575, 13578, 13661, 13663, 13729, 13762, 13805, 13825, 13886, 13889, 13956, 13964, 13978, 14007, 14031, 14072, 14192, 14232, 14258, 14284, 14436, 14480, 14499, 14539, 14572, 14584, 14592, 14598, 14599, 14659, 14709, 14722, 14741, 14902, 15187, 15230, 15259, 15401, 15616, 15737, 15746, 15761, 15810, 15926, 16075, 16268, 16283, 16307, 18671, 18685, 18725, 18872, 18924, 18972, 18979, 19128, 19135, 19213, 19316, 19324, 19416, 19504, 19588, 19596, 19674, 19734, 19843, 19882, 19974, 19996, 20012, 20040, 20076

In [69]:
len(suggestions)

1351

In [70]:
# Extracting out the end result information.
df_main = df_2[df_2.index.isin(suggestions)]

df_main.head()

,id,name,duration_ms,artists,danceability,energy,loudness,mode,speechiness,acousticness,...,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11,standardized_year,y_kmeans
3127,1BPybPVkDfUjFDvqG04l58,Breathe (feat. Sean Paul) - Rap Version,228000,0,0.0,0.09945,0.089363,0.1,0.001620,0.009600,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.142857,0
7547,6ea3gVkbrhghwfoio3NFpi,En cambio no,237440,0,0.0,0.05670,0.084797,0.1,0.001330,0.106000,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.380952,1
7624,0gPgdRfB4jdGrlyXS0Vx78,Khuda Jaane,333584,0,0.0,0.09480,0.085288,0.1,0.002055,0.078200,...,0.0,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.380952,2
7959,3BbGbnqvF2hJjbK3uCYq1M,Natasha - Ao Vivo,197933,0,0.0,0.14025,0.086345,0.1,0.003445,0.000842,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.380952,0
8198,0Tobkwkbgb8iZFt1zfnPby,Y.O.U. (Year Of Us),226960,0,0.0,0.06045,0.081802,0.0,0.001520,0.060400,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.428571,0


In [71]:
len(df_main) # There is one less for the last 3 features as the input song did not go through the extracted year's steps.
# df_final['standardized_year'].describe()

1225

In [72]:
df_main = df_main.iloc[1:]

df_main = df_main.drop('artists', axis=1)

id_artist_mapping = df.set_index('id')['artists'].to_dict()

# Add 'artist' column to df1 using the mapping
df_main['artists'] = df_main['id'].map(id_artist_mapping)
df_main.head()

,id,name,duration_ms,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,...,key_5,key_6,key_7,key_8,key_9,key_10,key_11,standardized_year,y_kmeans,artists
7547,6ea3gVkbrhghwfoio3NFpi,En cambio no,237440,0.0,0.05670,0.084797,0.1,0.001330,0.106000,0.000000,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.380952,1,['Laura Pausini']
7624,0gPgdRfB4jdGrlyXS0Vx78,Khuda Jaane,333584,0.0,0.09480,0.085288,0.1,0.002055,0.078200,0.000121,...,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.380952,2,"['Vishal-Shekhar', 'KK', 'Shilpa Rao']"
7959,3BbGbnqvF2hJjbK3uCYq1M,Natasha - Ao Vivo,197933,0.0,0.14025,0.086345,0.1,0.003445,0.000842,0.039250,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.380952,0,['Capital Inicial']
8198,0Tobkwkbgb8iZFt1zfnPby,Y.O.U. (Year Of Us),226960,0.0,0.06045,0.081802,0.0,0.001520,0.060400,0.000000,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.428571,0,['SHINee']
8212,5RNJtcXvXdTu8BqmFwXE8P,SHINee Girl,195880,0.0,0.13800,0.091862,0.0,0.010750,0.023000,0.000000,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.428571,2,['SHINee']


### Hard Category Filtering

In [73]:
# Keeping only the songs that match the input song's language.
df_filter = pd.DataFrame()

# Filter rows where the genre is present in the 'supersetted_genres' column
language_filter = df_main[df_main['language'].apply(lambda x: language_prority == x)]
# Concatenate filtered DataFrame with df_filter
df_filter = pd.concat([df_filter, language_filter])

In [74]:
# Keeping only the songs that match the input song's genre(s)
# Iterate over each genre in the priority list
for genre in genre_priority:
    # Filter rows where the genre is present in the 'supersetted_genres' column
    genre_filter = df_filter[df_main['supersetted_genres'].apply(lambda x: genre in x)]
    # Concatenate filtered DataFrame with df_filter
    df_filter = pd.concat([df_filter, genre_filter])

C:\Users\yukwa\AppData\Local\Temp\ipykernel_28468\1540286923.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  genre_filter = df_filter[df_main['supersetted_genres'].apply(lambda x: genre in x)]
C:\Users\yukwa\AppData\Local\Temp\ipykernel_28468\1540286923.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  genre_filter = df_filter[df_main['supersetted_genres'].apply(lambda x: genre in x)]
C:\Users\yukwa\AppData\Local\Temp\ipykernel_28468\1540286923.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  genre_filter = df_filter[df_main['supersetted_genres'].apply(lambda x: genre in x)]


In [75]:
prev_years = year_priority - 3
future_years = year_priority + 3

df_filter = df_filter[(df_filter['year'] >= prev_years) & (df_filter['year'] <= future_years)]

In [76]:
df_filter = df_filter[df_filter['y_kmeans'] == cluster_priority]

In [77]:
print(df_filter.shape)
df_filter.drop_duplicates(subset=["id"], keep='first', inplace=True)
print(df_filter.shape)

(101, 35)
(59, 35)


In [78]:
df_filter.iloc[:,1:19].head()

,name,duration_ms,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,year,language,langauge,song_id_name,genres_count,supersetted_genres
11799,Experience The Beyond - Official Iqon Anthem 2013,227239,0.0,0.13545,0.085719,0.0,0.002230,0.000756,2.057500e-02,0.00815,0.07500,0.068128,2015,en,en,7mi6ITB3GYsC5mBL5VplY2Experience The Beyond - ...,3,"[rawstyle, euphoric hardstyle]"
11952,7even - Radio Edit,252236,0.0,0.06180,0.079794,0.1,0.001965,0.021800,7.225000e-02,0.00570,0.00759,0.057609,2014,en,en,6YvZBHm6JuUr713pND1CZk7even - Radio Edit,5,[trance]
12741,Rather Be (feat. Jess Glynne),227833,0.0,0.08790,0.084744,0.1,0.001885,0.032400,5.075000e-07,0.00965,0.08235,0.054962,2014,en,en,3s4U7OHV7gnj42VV72eSZ6Rather Be (feat. Jess Gl...,10,"[edm, uk dance, uk pop, pop, uk pop]"
12934,Word of Mouth - 2014 Anthology Version; Remast...,225680,0.0,0.12855,0.081387,0.0,0.004640,0.007040,1.912500e-01,0.01190,0.07440,0.065224,2014,en,en,1VFIS4GVVUEKzjVf14pjVxWord of Mouth - 2014 Ant...,4,"[electric bass, jazz]"
13004,Cold Cold Man,230773,0.0,0.09915,0.087530,0.1,0.001315,0.001304,0.000000e+00,0.00488,0.07635,0.050422,2014,en,en,6RKGBcDtgzXx21zUjpeu6FCold Cold Man,7,"[la indie, rockabilly, rock]"


### Recommendations Output

In [79]:
# Generating recc_num of the matched songs at random.
def recommend(df_filter, recc_num):
    recs = []
    if len(df_filter) == 0:
        print("Sorry! There are no songs similar enough to " + input_song + "!")
    elif len(df_filter) < recc_num:
        recc_num = len(df_filter)
        print("Uh oh! We only found " + str(recc_num) + " recommendations!")
    print('Here are your recommendations!')
    for c in range(recc_num):
        recc = df_filter.sample().to_dict()
        name = list(recc['name'].values())[0]
        artists = (list(recc['artists'].values())[0])[1:len(list(recc['artists'].values())[0])-1]
        year = str(list(recc['year'].values())[0])
        id = str(list(recc['id'].values())[0])
        print(str(c + 1) + '. ' + name + ' by ' + artists + ' published in ' + year)
        df_filter = df_filter.drop(list(recc['id'])[0])
        recs.append((id, name, year))
    return recs
# May want to try a filter for repreventing duplicate songs in the list.
recs_for_user = recommend(df_filter, recc_num) # End Product

Here are your recommendations!
1. Hurts So Good by 'Astrid S' published in 2016
2. Be Real (feat. DeJ Loaf) by 'Kid Ink', 'DeJ Loaf' published in 2015
3. You Got Me Baby by 'Lushington' published in 2016
4. Adicto a Ti by 'Bacilos' published in 2018
5. Real Tears for Sale - Live in Port Zélande, 2009 by 'Marillion' published in 2018
6. Live For Tonight by 'Super8 & Tab', 'Jess Ball' published in 2020
7. Perfect (feat. Haris) - LUM!X Remix by 'Lucas & Steve', 'Haris', 'LUM!X' published in 2019
8. From the Chaff... (Mary, Mary EP 2012) by "Sunflo'er" published in 2017
9. All Or Nothing [ANR034] - Original Mix by 'Ben Gold', 'Christina Novelli' published in 2015
10. Thinking About It (Let It Go) - KVR Remix by 'Nathan Goshen', 'KVR' published in 2016
11. Rather Be (feat. Jess Glynne) by 'Clean Bandit', 'Jess Glynne' published in 2014
12. Both (feat. Drake) by 'Gucci Mane', 'Drake' published in 2016
13. Cringe by 'Matt Maeson' published in 2019
14. We're Coming Up by 'Julie Bergan' publish

In [80]:
user_id = sp.current_user()["id"]
song_uris = []
for song in recs_for_user:
    result = sp.track(song[0])
    print(result)
    try:
        uri = result["uri"]
        song_uris.append(uri)
    except IndexError:
        print(f"{song[0]} doesn't exist in Spotify. Skipped.")

playlist = sp.user_playlist_create(user=user_id, name=input_playlist, public=False)
sp.playlist_add_items(playlist_id=playlist["id"], items=song_uris)

{'album': {'album_type': 'single', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3AVfmawzu83sp94QW7CEGm'}, 'href': 'https://api.spotify.com/v1/artists/3AVfmawzu83sp94QW7CEGm', 'id': '3AVfmawzu83sp94QW7CEGm', 'name': 'Astrid S', 'type': 'artist', 'uri': 'spotify:artist:3AVfmawzu83sp94QW7CEGm'}], 'available_markets': ['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'DE', 'EC', 'EE', 'SV', 'FI', 'FR', 'GR', 'GT', 'HN', 'HK', 'HU', 'IS', 'IE', 'IT', 'LV', 'LT', 'LU', 'MY', 'MT', 'MX', 'NL', 'NZ', 'NI', 'NO', 'PA', 'PY', 'PE', 'PH', 'PL', 'PT', 'SG', 'SK', 'ES', 'SE', 'CH', 'TW', 'TR', 'UY', 'US', 'GB', 'AD', 'LI', 'MC', 'ID', 'JP', 'TH', 'VN', 'RO', 'IL', 'ZA', 'SA', 'AE', 'BH', 'QA', 'OM', 'KW', 'EG', 'MA', 'DZ', 'TN', 'LB', 'JO', 'PS', 'IN', 'BY', 'KZ', 'MD', 'UA', 'AL', 'BA', 'HR', 'ME', 'MK', 'RS', 'SI', 'KR', 'BD', 'PK', 'LK', 'GH', 'KE', 'NG', 'TZ', 'UG', 'AG', 'AM', 'BS', 'BB', 'BZ', 'BT', 'BW', 'BF', 'CV', 'CW',

{'snapshot_id': 'MixmZDEyMTI2MzRjOWNjNzY3YzVkMjViOTAwNjY3YWM3ZDBjZmJhZTlm'}